<a href="https://colab.research.google.com/github/JackCollins1991/GoogleColabs/blob/LogisticRegressionAssumptions/LogisticRegressionAssumptionsHelper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from copy import deepcopy
!pip install factor_analyzer

import pandas as pd
from factor_analyzer import FactorAnalyzer
from factor_analyzer.factor_analyzer import calculate_kmo
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
import statsmodels.api as sm

pd.set_option('display.max_columns', None)

In [5]:
class LogisticRegressionAssumptionsTestSummary:
  x = 1


In [10]:
def LogisticRegressionAssumptionsTest(predictors, dependant):
  ## http://www.sthda.com/english/articles/36-classification-methods-essentials/148-logistic-regression-assumptions-and-diagnostics-in-r/
  # Bar plots the number of cases of each type in the dependant variable
	TestClassBalance(dependant, positive)
	TestLinearityAssumption(predictors)
	TestInfluentialValues(predictors)
	TestMulticollinearity(predictors)
  return


def TestClassBalance(dependant, positive):

  return

def TestLinearityAssumption(predictors):

  return

def TestInfluentialValues(predictors):

  return

def TestMulticollinearity(predictors):

  return
